### TODO
- Download COCO
- Get some samples out of COCO val (figure out how to parse the annotations file)
- Run code at https://pyimagesearch.com/2021/08/02/pytorch-object-detection-with-pre-trained-networks/

### Roadmap
- Have SSD300 running
- Have Faster R-CNN running
- Have YOLOv3 running
- Get full outputs from SSD300, Faster R-CNN,. and YOLOv3
- 
- Generate adversarial examples for SSD300 and test on SSD300
- Generate adversarial examples for SSD300 and test on YOLOv3
- Generate adversarial examples with targeted loss and repeat experiments
- Notice any improvements?
- Make a transferability matrix
- Write a paper about it


In [2]:
from src.coco_eval import load_coco_dataset_in_fiftyone
import fiftyone as fo


def load_coco_dataset_in_fiftyone(dir: str, use_cached: bool = False, max_samples: int = None):
    if dir in fo.list_datasets():
        if use_cached:
            return fo.load_dataset(dir)
        
        fo.delete_dataset(dir)
    
    coco_dataset = fo.Dataset.from_dir(
        dataset_type = fo.types.COCODetectionDataset,
        data_path = f'{dir}/images/val2017',
        labels_path = f'{dir}/images/labels.json',
        include_id = True,
        label_field = '',
        max_samples = max_samples,
        name = dir
    )
    return coco_dataset

dataset = load_coco_dataset_in_fiftyone('data/coco', max_samples=10, use_cached=False)

 100% |███████████████████| 10/10 [61.0ms elapsed, 0s remaining, 163.9 samples/s]     


In [ ]:
import os, json

with open('data/coco/annotations/instances_val2017.json') as f:
    data = json.load(f)

In [ ]:
category_id_to_name = {}    # {1: 'person', 2: 'bicycle', 3: 'car', 4: 'motorcycle', ...}
image_filename_to_id = {}   # {'000000397133.jpg': 397133, '000000037777.jpg': 37777,  ...}
image_id_to_annotation = {}

idx_to_coco_id = {}
coco_labels = {0: 'background'}


for category in data['categories']:
    category_id_to_name[category['id']] = category['name']


for image in data['images']:
    image_filename_to_id[image['file_name']] = image['id']


# only add largest area's annotations to image (one image one annotation, the one with largest area)
for annotation in data['annotations']:
    image_id = annotation['image_id']

    # skip if area is non-max
    if image_id in image_id_to_annotation:
        if annotation['area'] < image_id_to_annotation[image_id]['area']:
            continue
            
    image_id_to_annotation[annotation['image_id']] = annotation


with open('data/coco/coco_labels.txt') as f:
    for i, line in enumerate(f.readlines()):
        data = line.split(',')
        id = int(data[0])               # stupid ID used
        name = data[2].strip()
        coco_labels[id] = name
        idx_to_coco_id[i] = id

n_classes = max(coco_labels.keys())

# fill non-existing cats to N/A
# for i in range(n_classes):
#     if coco_labels.get(i, -1) == -1:
#         coco_labels[i] = 'N/A'
    
print(idx_to_coco_id)

In [ ]:
coco_names = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

#### Image Selection
- Build conversion dictionaries
- Move some images to the 'data/coco/val' folder
- Build a dictionary of annotations. For each image in the folder:
    - Get its filename
    - Search in the images list the corresponding id
    - Search the annotations list for bbox and category

In [ ]:
annotations = {}

for fname in os.listdir('data/coco/val'):
    # fetch annotation
    image_id = image_filename_to_id[fname]
    annotation = image_id_to_annotation[image_id]

    # extract relevant data
    category_id = annotation['category_id']
    category_name = category_id_to_name[category_id]
    bbox = tuple(int(val) for val in annotation['bbox'])
    annotations[fname] = (category_name, bbox)

annotations;